In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir('/home/svcl-oowl/brandon/research/CVPR_2021_REFINE/sil_consistent_at_inference')
print(os.getcwd())

/home/svcl-oowl/brandon/research/CVPR_2021_REFINE/sil_consistent_at_inference


In [2]:
import pickle
import pprint

import torch

from utils import general_utils, datasets
from utils.datasets import gen_data_collate
from deformation.deformation_net_graph_convolutional_full import DeformationNetworkGraphConvolutionalFull
from deformation.multiview_semantic_discriminator_network import MultiviewSemanticDiscriminatorNetwork
from deformation.semantic_discriminator_net_points import PointsSemanticDiscriminatorNetwork
from deformation.forward_pass import batched_forward_pass, compute_sem_dis_logits
from utils.visualization_tools import save_tensor_img

In [15]:
cfg_path = "configs/test.yaml"

device = torch.device("cuda:0")
cfg = general_utils.load_config(cfg_path, "configs/default.yaml")
batch_size = cfg["semantic_dis_training"]["batch_size"]

# Test Datasets

In [16]:
generation_dataset = datasets.GenerationDataset(cfg)
generation_loader = torch.utils.data.DataLoader(generation_dataset, batch_size=batch_size, num_workers=1, shuffle=False, collate_fn=gen_data_collate, drop_last=True)


Loading cached generation dataset at caches/generation_dataset_cache_500.pt...


In [5]:
print(len(generation_dataset))
print(generation_dataset[0])
print(generation_dataset[0]["mesh_verts"].shape)

500
{'instance_name': '000_fake', 'mesh': <pytorch3d.structures.meshes.Meshes object at 0x7fce0b038240>, 'mesh_verts': tensor([[5.0947, 4.9838, 5.0988],
        [5.1497, 5.0246, 5.1155],
        [5.0561, 5.0058, 5.0963],
        ...,
        [5.1186, 4.9819, 5.0694],
        [5.0715, 4.9620, 5.1475],
        [5.1361, 4.9888, 5.0745]]), 'image': tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
      

In [17]:
real_dataset = datasets.RealDataset(cfg, device)
semantic_dis_loader = torch.utils.data.DataLoader(real_dataset, batch_size=batch_size, num_workers=1, shuffle=True)

Reusing previusly computed points assets...
Loading cached generation dataset at caches/real_dataset/points/data_adversarial_sphere_points_test_real_meshes/cache.pt...


In [7]:
print(real_dataset[0].shape)
print(len(real_dataset))

torch.Size([642, 3])
500


# Test Networks

In [12]:
deform_net = DeformationNetworkGraphConvolutionalFull(cfg, device)
deform_net.to(device)
semantic_dis_net = PointsSemanticDiscriminatorNetwork(cfg)
semantic_dis_net.to(device)

PointsSemanticDiscriminatorNetwork(
  (pointnet_discriminator): Sequential(
    (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): MaxPool1d(kernel_size=642, stride=642, padding=0, dilation=1, ceil_mode=False)
    (7): Flatten()
    (8): Linear(in_features=1024, out_features=512, bias=True)
    (9): LeakyReLU(negative_slope=0.01)
    (10): Linear(in_features=512, out_features=1, bias=True)
    (11): Sigmoid()
  )
)

In [19]:
for real_batch in semantic_dis_loader:
    #real_image_batch = real_batch["mv_images"].to(device)
    #print(real_image_batch.shape)
    print(real_batch.shape)
    real_batch = real_batch.to(device)
    pred_logits_real = semantic_dis_net(real_batch)
    print(pred_logits_real.shape)
    raise

torch.Size([4, 642, 3])
torch.Size([4, 1])


RuntimeError: No active exception to reraise

In [20]:
# TODO: sanity check these images
for gen_batch in generation_loader:
    _, deformed_meshes, _ = batched_forward_pass(cfg, device, deform_net, semantic_dis_net, gen_batch, compute_losses=False)
    pred_logits_fake, semantic_dis_debug_data = compute_sem_dis_logits(deformed_meshes, semantic_dis_net, device, cfg)
    print(pred_logits_fake.shape)
    raise

torch.Size([4, 1])


RuntimeError: No active exception to reraise

# Misc